In [3]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pandas.tseries.offsets import MonthEnd, MonthBegin
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')

In [4]:
cstat = pd.read_feather('compustat.feather')
cstat.reset_index(inplace=True)
cstat = cstat.rename(columns={'datadate': 'DATE', 'LPERMNO': 'PERMNO'}) 
cstat['DATE'] = pd.to_datetime(cstat['DATE'])
cstat['Month_Year'] = cstat['DATE'].dt.to_period('M')
# cstat.drop(['index'], axis = 1, inplace = True)
cstat

,DATE,PERMNO,datacqtr,datafqtr,atq,ibq,niq,saleq,seqq,iby,Month_Year
0,1983-09-30,10015,1983Q3,1983Q3,9.448,0.364,0.364,6.449,1.962,NaN,1983-09
1,1983-12-31,10015,1983Q4,1983Q4,14.080,0.249,0.249,8.166,7.823,NaN,1983-12
2,1984-03-31,10015,1984Q1,1984Q1,14.137,0.257,0.257,6.434,8.080,0.257,1984-03
3,1984-06-30,10015,1984Q2,1984Q2,15.152,0.425,0.425,7.559,8.505,NaN,1984-06
4,1984-09-30,10015,1984Q3,1984Q3,16.175,0.403,0.403,8.058,8.908,NaN,1984-09
...,...,...,...,...,...,...,...,...,...,...,...
1086862,2023-06-30,16161,2023Q2,2023Q2,16.709,-2.107,-2.107,0.000,9.072,-4.710,2023-06
1086863,2022-09-30,23209,2022Q3,2022Q3,NaN,332.320,332.320,2699.605,NaN,960.527,2022-09
1086864,2022-12-31,23209,2022Q4,2022Q4,41948.594,316.667,316.667,4079.260,19677.222,1277.194,2022-12
1086865,2023-03-31,23209,2023Q1,2023Q1,NaN,481.232,481.232,3693.981,NaN,481.232,2023-03


In [5]:
crsp = pd.read_feather('crsp.feather')
crsp.reset_index(inplace=True)
crsp = crsp.rename(columns={'date': 'DATE'}) 
crsp['DATE'] = pd.to_datetime(crsp['DATE'])
crsp['Month_Year'] = crsp['DATE'].dt.to_period('M')

In [7]:
# crsp.drop(['index'], axis = 1, inplace = True)
crsp

,DATE,PERMNO,SHRCD,PERMCO,DIVAMT,PRC,VOL,RET,BID,ASK,SHROUT,OPENPRC,RETX,sprtrn,Month_Year
0,1986-01-07,10000,10,7952,NaN,-2.56250,1000.0,NaN,NaN,NaN,3680.0,NaN,NaN,0.014954,1986-01
1,1986-01-08,10000,10,7952,NaN,-2.50000,12800.0,-0.024390,NaN,NaN,3680.0,NaN,-0.024390,-0.027268,1986-01
2,1986-01-09,10000,10,7952,NaN,-2.50000,1400.0,0.000000,NaN,NaN,3680.0,NaN,0.000000,-0.008944,1986-01
3,1986-01-10,10000,10,7952,NaN,-2.50000,8500.0,0.000000,NaN,NaN,3680.0,NaN,0.000000,-0.000728,1986-01
4,1986-01-13,10000,10,7952,NaN,-2.62500,5450.0,0.050000,NaN,NaN,3680.0,NaN,0.050000,0.003690,1986-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57415487,2023-06-26,93436,11,53453,NaN,241.05000,179758166.0,-0.060600,241.02000,241.05000,3169504.0,250.06500,-0.060600,-0.004487,2023-06
57415488,2023-06-27,93436,11,53453,NaN,250.21001,164827712.0,0.038000,250.20000,250.21001,3169504.0,243.24001,0.038000,0.011456,2023-06
57415489,2023-06-28,93436,11,53453,NaN,256.23999,159593987.0,0.024100,256.39001,256.41000,3169504.0,249.70000,0.024100,-0.000354,2023-06
57415490,2023-06-29,93436,11,53453,NaN,257.50000,131177205.0,0.004917,257.51999,257.54001,3169504.0,258.03000,0.004917,0.004474,2023-06


In [8]:
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', skiprows=5,
                 names=['DATE','Mkt-RF','SMB','HML','RF'])[:-1]
ff['DATE'] = pd.to_datetime(ff['DATE'], format='%Y%m%d')
ff = ff.set_index('DATE')
ff

,Mkt-RF,SMB,HML,RF
DATE,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2023-09-25,0.39,-0.11,0.35,0.021
2023-09-26,-1.46,0.39,-0.50,0.021
2023-09-27,0.13,0.69,0.12,0.021


# 1. Data Cleaning

## 1.1 CRSP Data Cleaning

In [9]:
crsp['RETX'].interpolate(method='linear', inplace=True)
crsp['RET'].interpolate(method='linear', inplace=True)

In [10]:
#making PRC into absolute value
crsp['PRC']  = np.abs(crsp['PRC'])

In [11]:
#drop the rows with vol == 0
crsp = crsp.loc[crsp['VOL'] > 0].copy()

In [12]:
#compute market value as MV
crsp['MV'] = crsp['SHROUT'] * crsp['PRC']
crsp = crsp[crsp['MV'] > 50000]
crsp = crsp[crsp['VOL'] > 0]

In [13]:
#drop useless columns
crsp.drop(['DIVAMT','PERMCO','SHRCD','VOL','BID','ASK','sprtrn'], axis=1, inplace=True)

In [14]:
crsp.set_index(['PERMNO','Month_Year'], inplace=True)
crsp.sort_index(inplace=True)

In [15]:
# crsp['manualRet'] = crsp['PRC'].groupby('PERMNO').pct_change()
# crsp['RET'] = crsp['RET'].fillna(crsp['manualRet'])

In [16]:
#compute volatility by calculating the standard deviation of RETX(the return with dividends)
# crsp['lag252 RETX'] = crsp['RET'].groupby('PERMNO').shift(252)
crsp['volatility'] = crsp['RET'].groupby('PERMNO').rolling(90).std().droplevel(0)
crsp

DATE        PRC       RET     SHROUT    OPENPRC  \
PERMNO Month_Year                                                         
10001  2010-04    2010-04-27   11.61000  0.045946     4361.0   11.13000   
       2010-04    2010-04-29   11.52000  0.024920     4361.0   11.30000   
       2010-04    2010-04-30   11.39000 -0.011285     6070.0   11.98000   
       2010-05    2010-05-03   11.80000  0.035996     6070.0   11.65000   
       2010-05    2010-05-04   11.56000 -0.020339     6070.0   11.50000   
...                      ...        ...       ...        ...        ...   
93436  2023-06    2023-06-26  241.05000 -0.060600  3169504.0  250.06500   
       2023-06    2023-06-27  250.21001  0.038000  3169504.0  243.24001   
       2023-06    2023-06-28  256.23999  0.024100  3169504.0  249.70000   
       2023-06    2023-06-29  257.50000  0.004917  3169504.0  258.03000   
       2023-06    2023-06-30  261.76999  0.016582  3169504.0  260.60001   

                       RETX            MV  volatility  
PERMNO Month_Year                                      
10001  2010-04     0.045946  5.063121e+04         NaN  
       2010-04     0.024920  5.023872e+04         NaN  
       2010-04    -0.011285  6.913730e+04         NaN  
       2010-05     0.035996  7.162600e+04         NaN  
       2010-05    -0.020339  7.016920e+04         NaN  
...                     ...           ...         ...  
93436  2023-06    -0.060600  7.640089e+08    0.032280  
       2023-06     0.038000  7.930416e+08    0.032421  
       2023-06     0.024100  8.121537e+08    0.031880  
       2023-06     0.004917  8.161473e+08    0.031742  
       2023-06     0.016582  8.296810e+08    0.031219  

[35643803 rows x 8 columns]

## 1.2 COMPUSTAT Data Cleaning

In [17]:
cstat

,DATE,PERMNO,datacqtr,datafqtr,atq,ibq,niq,saleq,seqq,iby,Month_Year
0,1983-09-30,10015,1983Q3,1983Q3,9.448,0.364,0.364,6.449,1.962,NaN,1983-09
1,1983-12-31,10015,1983Q4,1983Q4,14.080,0.249,0.249,8.166,7.823,NaN,1983-12
2,1984-03-31,10015,1984Q1,1984Q1,14.137,0.257,0.257,6.434,8.080,0.257,1984-03
3,1984-06-30,10015,1984Q2,1984Q2,15.152,0.425,0.425,7.559,8.505,NaN,1984-06
4,1984-09-30,10015,1984Q3,1984Q3,16.175,0.403,0.403,8.058,8.908,NaN,1984-09
...,...,...,...,...,...,...,...,...,...,...,...
1086862,2023-06-30,16161,2023Q2,2023Q2,16.709,-2.107,-2.107,0.000,9.072,-4.710,2023-06
1086863,2022-09-30,23209,2022Q3,2022Q3,NaN,332.320,332.320,2699.605,NaN,960.527,2022-09
1086864,2022-12-31,23209,2022Q4,2022Q4,41948.594,316.667,316.667,4079.260,19677.222,1277.194,2022-12
1086865,2023-03-31,23209,2023Q1,2023Q1,NaN,481.232,481.232,3693.981,NaN,481.232,2023-03


In [18]:
#rename LPERMNO to PERMNO, set date form.
# cstat.rename(columns={"LPERMNO":"PERMNO"}, inplace=True)
cstat['DATE'] = cstat['DATE']

In [19]:
#retain useful columns "ibq" and 'seqq', because data is quarterly
cstat.drop(['datacqtr','datafqtr','saleq','iby'], axis=1, inplace=True)

In [20]:
#rename ibq to IB and seqq to SEQ
cstat.rename(columns = {"ibq":'IB', 'seqq':'SEQ', 'atq' :'ASSET', 'niq' : 'NI'}, inplace = True)

In [21]:
#set index and sort by index
cstat.set_index(['PERMNO','Month_Year'], inplace=True)
cstat.sort_index(inplace=True)
cstat

DATE      ASSET        IB        NI        SEQ
PERMNO Month_Year                                                     
10000  1986-01    1986-01-31      1.265    -0.132    -0.132      0.689
       1986-04    1986-04-30      1.284    -0.484    -0.484      0.578
       1986-07    1986-07-31      1.311    -0.132    -0.132      0.289
       1986-10    1986-10-31      2.115     0.018     0.018      0.418
       1987-01    1987-01-31        NaN    -0.131    -0.131        NaN
...                      ...        ...       ...       ...        ...
93436  2022-09    2022-09-30  74426.000  3292.000  3292.000  39851.000
       2022-12    2022-12-31  82338.000  3687.000  3687.000  44704.000
       2023-03    2023-03-31  86833.000  2513.000  2513.000  48054.000
       2023-06    2023-06-30  90591.000  2703.000  2703.000  51130.000
       2023-09    2023-09-30  93941.000  1853.000  1853.000  53466.000

[1086867 rows x 5 columns]

In [22]:
#sorting by data: ascending for PERMNO and DATE and descending by MV and SEQ
crsp = crsp.sort_values(by = ['PERMNO','DATE','MV'], ascending = [True, True, False])
cstat  = cstat.sort_values(by = ['PERMNO','DATE','SEQ'], ascending = [True, True, False])

In [23]:
#merge crsp and cstat
crsp = crsp.merge(cstat[['IB','SEQ', 'ASSET', 'NI']], how='left', on=['PERMNO','Month_Year'])
crsp

DATE        PRC       RET     SHROUT    OPENPRC  \
PERMNO Month_Year                                                         
10001  2010-04    2010-04-27   11.61000  0.045946     4361.0   11.13000   
       2010-04    2010-04-29   11.52000  0.024920     4361.0   11.30000   
       2010-04    2010-04-30   11.39000 -0.011285     6070.0   11.98000   
       2010-05    2010-05-03   11.80000  0.035996     6070.0   11.65000   
       2010-05    2010-05-04   11.56000 -0.020339     6070.0   11.50000   
...                      ...        ...       ...        ...        ...   
93436  2023-06    2023-06-26  241.05000 -0.060600  3169504.0  250.06500   
       2023-06    2023-06-27  250.21001  0.038000  3169504.0  243.24001   
       2023-06    2023-06-28  256.23999  0.024100  3169504.0  249.70000   
       2023-06    2023-06-29  257.50000  0.004917  3169504.0  258.03000   
       2023-06    2023-06-30  261.76999  0.016582  3169504.0  260.60001   

                       RETX            MV  volatility      IB      SEQ  \
PERMNO Month_Year                                                        
10001  2010-04     0.045946  5.063121e+04         NaN     NaN      NaN   
       2010-04     0.024920  5.023872e+04         NaN     NaN      NaN   
       2010-04    -0.011285  6.913730e+04         NaN     NaN      NaN   
       2010-05     0.035996  7.162600e+04         NaN     NaN      NaN   
       2010-05    -0.020339  7.016920e+04         NaN     NaN      NaN   
...                     ...           ...         ...     ...      ...   
93436  2023-06    -0.060600  7.640089e+08    0.032280  2703.0  51130.0   
       2023-06     0.038000  7.930416e+08    0.032421  2703.0  51130.0   
       2023-06     0.024100  8.121537e+08    0.031880  2703.0  51130.0   
       2023-06     0.004917  8.161473e+08    0.031742  2703.0  51130.0   
       2023-06     0.016582  8.296810e+08    0.031219  2703.0  51130.0   

                     ASSET      NI  
PERMNO Month_Year                   
10001  2010-04         NaN     NaN  
       2010-04         NaN     NaN  
       2010-04         NaN     NaN  
       2010-05         NaN     NaN  
       2010-05         NaN     NaN  
...                    ...     ...  
93436  2023-06     90591.0  2703.0  
       2023-06     90591.0  2703.0  
       2023-06     90591.0  2703.0  
       2023-06     90591.0  2703.0  
       2023-06     90591.0  2703.0  

[35648171 rows x 12 columns]

In [24]:
# crsp.rename(columns = {'IB':'profitability'}, inplace = True)
crsp['profitability'] = crsp['IB'] / crsp['ASSET']

In [25]:
crsp[['IB', 'SEQ', 'ASSET', 'NI', 'profitability']] = crsp[['IB', 'SEQ', 'ASSET',
                                                            'NI', 'profitability']].ffill()

In [26]:
crsp.reset_index(inplace=True)
crsp.set_index(['PERMNO', 'DATE'], inplace=True)

In [27]:
crsp

Month_Year        PRC       RET     SHROUT    OPENPRC  \
PERMNO DATE                                                               
10001  2010-04-27    2010-04   11.61000  0.045946     4361.0   11.13000   
       2010-04-29    2010-04   11.52000  0.024920     4361.0   11.30000   
       2010-04-30    2010-04   11.39000 -0.011285     6070.0   11.98000   
       2010-05-03    2010-05   11.80000  0.035996     6070.0   11.65000   
       2010-05-04    2010-05   11.56000 -0.020339     6070.0   11.50000   
...                      ...        ...       ...        ...        ...   
93436  2023-06-26    2023-06  241.05000 -0.060600  3169504.0  250.06500   
       2023-06-27    2023-06  250.21001  0.038000  3169504.0  243.24001   
       2023-06-28    2023-06  256.23999  0.024100  3169504.0  249.70000   
       2023-06-29    2023-06  257.50000  0.004917  3169504.0  258.03000   
       2023-06-30    2023-06  261.76999  0.016582  3169504.0  260.60001   

                       RETX            MV  volatility      IB      SEQ  \
PERMNO DATE                                                              
10001  2010-04-27  0.045946  5.063121e+04         NaN     NaN      NaN   
       2010-04-29  0.024920  5.023872e+04         NaN     NaN      NaN   
       2010-04-30 -0.011285  6.913730e+04         NaN     NaN      NaN   
       2010-05-03  0.035996  7.162600e+04         NaN     NaN      NaN   
       2010-05-04 -0.020339  7.016920e+04         NaN     NaN      NaN   
...                     ...           ...         ...     ...      ...   
93436  2023-06-26 -0.060600  7.640089e+08    0.032280  2703.0  51130.0   
       2023-06-27  0.038000  7.930416e+08    0.032421  2703.0  51130.0   
       2023-06-28  0.024100  8.121537e+08    0.031880  2703.0  51130.0   
       2023-06-29  0.004917  8.161473e+08    0.031742  2703.0  51130.0   
       2023-06-30  0.016582  8.296810e+08    0.031219  2703.0  51130.0   

                     ASSET      NI  profitability  
PERMNO DATE                                        
10001  2010-04-27      NaN     NaN            NaN  
       2010-04-29      NaN     NaN            NaN  
       2010-04-30      NaN     NaN            NaN  
       2010-05-03      NaN     NaN            NaN  
       2010-05-04      NaN     NaN            NaN  
...                    ...     ...            ...  
93436  2023-06-26  90591.0  2703.0       0.029837  
       2023-06-27  90591.0  2703.0       0.029837  
       2023-06-28  90591.0  2703.0       0.029837  
       2023-06-29  90591.0  2703.0       0.029837  
       2023-06-30  90591.0  2703.0       0.029837  

[35648171 rows x 13 columns]

## 1.3 F-F_Research_Data_daily Data Cleaning

In [28]:
ff['Mkt-RF']=ff['Mkt-RF']/100
ff['SMB']=ff['SMB']/100
ff['HML']=ff['HML']/100
ff['RF']=ff['RF']/100

In [29]:
ff

,Mkt-RF,SMB,HML,RF
DATE,,,,
1926-07-01,0.0010,-0.0025,-0.0027,0.00009
1926-07-02,0.0045,-0.0033,-0.0006,0.00009
1926-07-06,0.0017,0.0030,-0.0039,0.00009
1926-07-07,0.0009,-0.0058,0.0002,0.00009
1926-07-08,0.0021,-0.0038,0.0019,0.00009
...,...,...,...,...
2023-09-25,0.0039,-0.0011,0.0035,0.00021
2023-09-26,-0.0146,0.0039,-0.0050,0.00021
2023-09-27,0.0013,0.0069,0.0012,0.00021


## 2. Strategies

In [30]:
df = crsp.copy()
df.reset_index(inplace=True)
df.set_index(['DATE'], inplace=True)
df = df.merge(ff[['Mkt-RF', 'SMB', 'HML', 'RF']], how='left', on='DATE')
df.reset_index(inplace=True)
df.set_index(['PERMNO', 'DATE'], inplace=True)
df.drop(columns=['OPENPRC', 'IB', 'ASSET'], inplace=True)
df

Month_Year        PRC       RET     SHROUT      RETX  \
PERMNO DATE                                                              
10001  2010-04-27    2010-04   11.61000  0.045946     4361.0  0.045946   
       2010-04-29    2010-04   11.52000  0.024920     4361.0  0.024920   
       2010-04-30    2010-04   11.39000 -0.011285     6070.0 -0.011285   
       2010-05-03    2010-05   11.80000  0.035996     6070.0  0.035996   
       2010-05-04    2010-05   11.56000 -0.020339     6070.0 -0.020339   
...                      ...        ...       ...        ...       ...   
93436  2023-06-26    2023-06  241.05000 -0.060600  3169504.0 -0.060600   
       2023-06-27    2023-06  250.21001  0.038000  3169504.0  0.038000   
       2023-06-28    2023-06  256.23999  0.024100  3169504.0  0.024100   
       2023-06-29    2023-06  257.50000  0.004917  3169504.0  0.004917   
       2023-06-30    2023-06  261.76999  0.016582  3169504.0  0.016582   

                             MV  volatility      SEQ      NI  profitability  \
PERMNO DATE                                                                   
10001  2010-04-27  5.063121e+04         NaN      NaN     NaN            NaN   
       2010-04-29  5.023872e+04         NaN      NaN     NaN            NaN   
       2010-04-30  6.913730e+04         NaN      NaN     NaN            NaN   
       2010-05-03  7.162600e+04         NaN      NaN     NaN            NaN   
       2010-05-04  7.016920e+04         NaN      NaN     NaN            NaN   
...                         ...         ...      ...     ...            ...   
93436  2023-06-26  7.640089e+08    0.032280  51130.0  2703.0       0.029837   
       2023-06-27  7.930416e+08    0.032421  51130.0  2703.0       0.029837   
       2023-06-28  8.121537e+08    0.031880  51130.0  2703.0       0.029837   
       2023-06-29  8.161473e+08    0.031742  51130.0  2703.0       0.029837   
       2023-06-30  8.296810e+08    0.031219  51130.0  2703.0       0.029837   

                   Mkt-RF     SMB     HML       RF  
PERMNO DATE                                         
10001  2010-04-27 -0.0234  0.0012 -0.0135  0.00001  
       2010-04-29  0.0134  0.0043  0.0056  0.00001  
       2010-04-30 -0.0172 -0.0097 -0.0088  0.00001  
       2010-05-03  0.0136  0.0062  0.0060  0.00001  
       2010-05-04 -0.0250 -0.0056 -0.0092  0.00001  
...                   ...     ...     ...      ...  
93436  2023-06-26 -0.0048 -0.0032  0.0130  0.00019  
       2023-06-27  0.0121  0.0047 -0.0052  0.00019  
       2023-06-28  0.0004  0.0054 -0.0042  0.00019  
       2023-06-29  0.0044  0.0052  0.0076  0.00019  
       2023-06-30  0.0114 -0.0063 -0.0077  0.00019  

[35648171 rows x 14 columns]

In [31]:
#calculate P/B ratio
df['lag SEQ'] = df['SEQ'].groupby('PERMNO').shift(60)
df['PB'] = df['MV'] / df['lag SEQ'] / 1000
df['lag profitability'] = df['profitability'].groupby('PERMNO').shift(60)

df['lag NI'] = df['NI'].groupby('PERMNO').shift(60)

df['PE'] = df['MV'] / df['lag NI'] / 1000

In [32]:
df.head()

Month_Year    PRC       RET  SHROUT      RETX        MV  \
PERMNO DATE                                                                 
10001  2010-04-27    2010-04  11.61  0.045946  4361.0  0.045946  50631.21   
       2010-04-29    2010-04  11.52  0.024920  4361.0  0.024920  50238.72   
       2010-04-30    2010-04  11.39 -0.011285  6070.0 -0.011285  69137.30   
       2010-05-03    2010-05  11.80  0.035996  6070.0  0.035996  71626.00   
       2010-05-04    2010-05  11.56 -0.020339  6070.0 -0.020339  70169.20   

                   volatility  SEQ  NI  profitability  Mkt-RF     SMB     HML  \
PERMNO DATE                                                                     
10001  2010-04-27         NaN  NaN NaN            NaN -0.0234  0.0012 -0.0135   
       2010-04-29         NaN  NaN NaN            NaN  0.0134  0.0043  0.0056   
       2010-04-30         NaN  NaN NaN            NaN -0.0172 -0.0097 -0.0088   
       2010-05-03         NaN  NaN NaN            NaN  0.0136  0.0062  0.0060   
       2010-05-04         NaN  NaN NaN            NaN -0.0250 -0.0056 -0.0092   

                        RF  lag SEQ  PB  lag profitability  lag NI  PE  
PERMNO DATE                                                             
10001  2010-04-27  0.00001      NaN NaN                NaN     NaN NaN  
       2010-04-29  0.00001      NaN NaN                NaN     NaN NaN  
       2010-04-30  0.00001      NaN NaN                NaN     NaN NaN  
       2010-05-03  0.00001      NaN NaN                NaN     NaN NaN  
       2010-05-04  0.00001      NaN NaN                NaN     NaN NaN

### Rank on P/B, profitability, and volatility. Use the rank as score and select the lowest scoring (high PB, high profitability, low volatilaty) stocks. Construct a portfolio of the top 1000 everyday

# 3 Implementation

# 3.1 In Sample Results

In [33]:
df_ins = df.iloc[df.index.get_level_values('DATE') <= '2012-12-31']
df_oos = df.iloc[df.index.get_level_values('DATE') >'2012-12-31']

In [34]:
df_ins.shape

(26886935, 19)

In [35]:
df_oos.shape

(8761236, 19)

In [36]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)
df_ins['rank'] = df_ins['growth rank'] + df_ins['profitability rank'] + df_ins['volatility rank']

# Growth + low volatility
# df['rank'] = df['growth rank'] + df['volatility rank']
# Low overall value
df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port = top1000['retp'].groupby('DATE').sum()

In [37]:
port

DATE
1980-05-06   -0.003079
1980-05-07    0.022756
1980-05-08   -0.007179
1980-05-09   -0.012351
1980-05-12    0.000320
                ...   
2012-12-24   -0.002033
2012-12-26   -0.005697
2012-12-27   -0.000500
2012-12-28   -0.010477
2012-12-31    0.016462
Name: retp, Length: 8238, dtype: float64

In [38]:
port = pd.DataFrame(port)
port = port.merge(ff, how = 'left', on = 'DATE')
port['Excess Return'] = port['retp'] - port['RF']
port

,retp,Mkt-RF,SMB,HML,RF,Excess Return
DATE,,,,,,
1980-05-06,-0.003079,0.0014,0.0049,-0.0006,0.00038,-0.003459
1980-05-07,0.022756,0.0087,-0.0003,0.0061,0.00038,0.022376
1980-05-08,-0.007179,-0.0071,0.0067,0.0017,0.00038,-0.007559
1980-05-09,-0.012351,-0.0118,0.0089,0.0014,0.00038,-0.012731
1980-05-12,0.000320,-0.0002,0.0003,-0.0003,0.00038,-0.000060
...,...,...,...,...,...,...
2012-12-24,-0.002033,-0.0024,-0.0019,-0.0002,0.00001,-0.002043
2012-12-26,-0.005697,-0.0054,-0.0019,0.0031,0.00001,-0.005707
2012-12-27,-0.000500,-0.0011,0.0000,-0.0013,0.00001,-0.000510


In [39]:
from sklearn.linear_model import LinearRegression

ffmodel = LinearRegression()
X = port[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
annual_alpha

0.12171767480665832

In [40]:
annual_ret = port['retp'].mean()*252
annual_ret

0.2280863060733869

In [41]:
annual_excess_ret = port['Excess Return'].mean()*252
annual_excess_ret

0.18026045028314655

In [42]:
annual_std = port['Excess Return'].std()*np.sqrt(252)
annual_std

0.16477150863461362

In [43]:
annual_sr = annual_excess_ret/annual_std
annual_sr

1.0940025479943876

# 3.2 Change Specification from PB to PE

In [44]:
df_ins['PE rank'] = df_ins['PE'].groupby('DATE').rank(ascending=False)
df_ins['rankWithPE'] = df_ins['PE rank'] + df_ins['profitability rank'] + df_ins['volatility rank']

# Growth + low volatility
# Low overall value
df_ins['rank'] = df_ins['rankWithPE'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
portPE = top1000['retp'].groupby('DATE').sum()

In [45]:
portPE

DATE
1980-05-06   -0.003079
1980-05-07    0.022756
1980-05-08   -0.006326
1980-05-09   -0.011581
1980-05-12    0.001265
                ...   
2012-12-24   -0.002432
2012-12-26   -0.006073
2012-12-27   -0.000477
2012-12-28   -0.010456
2012-12-31    0.015928
Name: retp, Length: 8238, dtype: float64

In [46]:
portPE = pd.DataFrame(portPE)
portPE = portPE.merge(ff, how = 'left', on = 'DATE')
portPE['Excess Return'] = portPE['retp'] - portPE['RF']
portPE

,retp,Mkt-RF,SMB,HML,RF,Excess Return
DATE,,,,,,
1980-05-06,-0.003079,0.0014,0.0049,-0.0006,0.00038,-0.003459
1980-05-07,0.022756,0.0087,-0.0003,0.0061,0.00038,0.022376
1980-05-08,-0.006326,-0.0071,0.0067,0.0017,0.00038,-0.006706
1980-05-09,-0.011581,-0.0118,0.0089,0.0014,0.00038,-0.011961
1980-05-12,0.001265,-0.0002,0.0003,-0.0003,0.00038,0.000885
...,...,...,...,...,...,...
2012-12-24,-0.002432,-0.0024,-0.0019,-0.0002,0.00001,-0.002442
2012-12-26,-0.006073,-0.0054,-0.0019,0.0031,0.00001,-0.006083
2012-12-27,-0.000477,-0.0011,0.0000,-0.0013,0.00001,-0.000487


In [47]:
from sklearn.linear_model import LinearRegression

ffmodel = LinearRegression()
X = portPE[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(portPE['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
annual_alpha

0.11873641579753064

In [48]:
annual_ret = portPE['retp'].mean()*252
annual_ret

0.22355055480647973

In [49]:
annual_excess_ret = portPE['Excess Return'].mean()*252
annual_excess_ret

0.17572469901623936

In [50]:
annual_std = portPE['Excess Return'].std()*np.sqrt(252)
annual_std

0.15908139911991503

In [51]:
annual_sr = annual_excess_ret/annual_std
annual_sr

1.1046212818619898

# 3.3 Change Specification from 90-day volatility to 252-day volatility

In [52]:
df_ins['volatility252'] = df_ins['RET'].groupby('PERMNO').rolling(252).std().droplevel(0)

df_ins['volatility252 rank'] = df_ins['volatility252'].groupby('DATE').rank(ascending=True)

df_ins['rank252'] = df_ins['growth rank'] + df_ins['profitability rank'] + df_ins['volatility252 rank']

# Growth + low volatility
# df['rank'] = df['growth rank'] + df['volatility rank']
# Low overall value
df_ins['rank252'] = df_ins['rank252'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank252'] < 1000]
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port252vol = top1000['retp'].groupby('DATE').sum()

In [53]:
port252vol = pd.DataFrame(port252vol)
port252vol = port252vol.merge(ff, how = 'left', on = 'DATE')
port252vol['Excess Return'] = port252vol['retp'] - port252vol['RF']
port252vol

,retp,Mkt-RF,SMB,HML,RF,Excess Return
DATE,,,,,,
1980-11-28,0.000000,0.0027,0.0001,0.0003,0.00053,-0.000530
1980-12-01,-0.027523,-0.0221,0.0091,0.0035,0.00059,-0.028113
1980-12-02,-0.009434,-0.0030,-0.0037,0.0026,0.00059,-0.010024
1980-12-03,0.000000,-0.0007,0.0056,-0.0035,0.00059,-0.000590
1980-12-04,0.009524,-0.0012,0.0057,-0.0019,0.00059,0.008934
...,...,...,...,...,...,...
2012-12-24,-0.002145,-0.0024,-0.0019,-0.0002,0.00001,-0.002155
2012-12-26,-0.005536,-0.0054,-0.0019,0.0031,0.00001,-0.005546
2012-12-27,-0.000516,-0.0011,0.0000,-0.0013,0.00001,-0.000526


In [54]:
ffmodel = LinearRegression()
X = port252vol[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port252vol['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
annual_alpha

0.11842910107187722

In [55]:
annual_ret = port252vol['retp'].mean()*252
annual_ret

0.21723727631393916

In [56]:
annual_excess_ret = port252vol['Excess Return'].mean()*252
annual_excess_ret

0.17013895883401328

In [57]:
annual_std = port252vol['Excess Return'].std()*np.sqrt(252)
annual_std

0.16734459922317865

In [58]:
annual_sr = annual_excess_ret/annual_std
annual_sr

1.0166982359980912

# 3.4 Combine two different specifications

In [59]:
df_ins['rank252PE'] = df_ins['PE rank'] + df_ins['profitability rank'] + df_ins['volatility252 rank']
df_ins['rank252PE'] = df_ins['rank252PE'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank252PE'] < 1000]
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port2c = top1000['retp'].groupby('DATE').sum()

In [60]:
port2c = pd.DataFrame(port2c)
port2c = port2c.merge(ff, how = 'left', on = 'DATE')
port2c['Excess Return'] = port2c['retp'] - port2c['RF']
port2c

,retp,Mkt-RF,SMB,HML,RF,Excess Return
DATE,,,,,,
1980-11-28,0.000000,0.0027,0.0001,0.0003,0.00053,-0.000530
1980-12-01,-0.027523,-0.0221,0.0091,0.0035,0.00059,-0.028113
1980-12-02,-0.009434,-0.0030,-0.0037,0.0026,0.00059,-0.010024
1980-12-03,0.000000,-0.0007,0.0056,-0.0035,0.00059,-0.000590
1980-12-04,0.009524,-0.0012,0.0057,-0.0019,0.00059,0.008934
...,...,...,...,...,...,...
2012-12-24,-0.002352,-0.0024,-0.0019,-0.0002,0.00001,-0.002362
2012-12-26,-0.005994,-0.0054,-0.0019,0.0031,0.00001,-0.006004
2012-12-27,-0.000497,-0.0011,0.0000,-0.0013,0.00001,-0.000507


In [61]:
ffmodel = LinearRegression()
X = port2c[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port2c['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
annual_alpha

0.11419013301043805

In [62]:
annual_ret = port2c['retp'].mean()*252
annual_ret

0.21195315947461085

In [63]:
annual_excess_ret = port2c['Excess Return'].mean()*252
annual_excess_ret

0.16485484199468495

In [64]:
annual_std = port2c['Excess Return'].std()*np.sqrt(252)
annual_std

0.1630289386589016

In [65]:
annual_sr = annual_excess_ret/annual_std
annual_sr

1.0111998725551639

## 3.5 Out of Sample Results

In [66]:
# High P/B ratio
df_oos['growth rank'] = df_oos['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_oos['profitability rank'] = df_oos['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_oos['volatility rank'] = df_oos['volatility'].groupby('DATE').rank(ascending=True)
df_oos['rank'] = df_oos['growth rank'] + df_oos['profitability rank'] + df_oos['volatility rank']

# Growth + low volatility
# df['rank'] = df['growth rank'] + df['volatility rank']
# Low overall value
df_oos['rank'] = df_oos['rank'].groupby('DATE').rank()
top1000 = df_oos.loc[df_oos['rank'] < 1000]
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port = top1000['retp'].groupby('DATE').sum()

In [67]:
port = pd.DataFrame(port)
port = port.merge(ff, how = 'left', on = 'DATE')
port['Excess Return'] = port['retp'] - port['RF']
port

,retp,Mkt-RF,SMB,HML,RF,Excess Return
DATE,,,,,,
2013-01-02,0.025285,0.0262,0.0015,0.0038,0.00000,0.025285
2013-01-03,-0.001713,-0.0014,0.0011,0.0004,0.00000,-0.001713
2013-01-04,0.003535,0.0055,0.0012,0.0036,0.00000,0.003535
2013-01-07,-0.003107,-0.0031,-0.0010,-0.0035,0.00000,-0.003107
2013-01-08,-0.001950,-0.0027,0.0005,0.0000,0.00000,-0.001950
...,...,...,...,...,...,...
2023-06-26,-0.005295,-0.0048,-0.0032,0.0130,0.00019,-0.005485
2023-06-27,0.012024,0.0121,0.0047,-0.0052,0.00019,0.011834
2023-06-28,-0.000117,0.0004,0.0054,-0.0042,0.00019,-0.000307


In [68]:
from sklearn.linear_model import LinearRegression

ffmodel = LinearRegression()
X = portPE[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(portPE['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
annual_alpha

0.11873641579753064

In [69]:
annual_ret = port['retp'].mean()*252
annual_ret

0.21000306614868805

In [70]:
annual_excess_ret = port['Excess Return'].mean()*252
annual_excess_ret

0.201504504453003

In [71]:
annual_std = port['Excess Return'].std()*np.sqrt(252)
annual_std

0.16979536246485621

In [72]:
annual_sr = annual_excess_ret/annual_std
annual_sr

1.1867491639809058

# 4 Testing: Comparison between strategies

# 4.1 Case 1 Growth + Profit

In [73]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)


# RANKING
df_ins['rank'] = df_ins['growth rank'] + df_ins['profitability rank']
df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]

#Get weight and return data
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port_test1 = top1000['retp'].groupby('DATE').sum()
port_test1 = pd.DataFrame(port_test1)
port_test1 = port_test1.merge(ff, how = 'left', on = 'DATE')
port_test1['Excess Return'] = port_test1['retp'] - port_test1['RF']


from sklearn.linear_model import LinearRegression
ffmodel = LinearRegression()
X = port_test1[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port_test1['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
print(annual_alpha)

#Find performances
annual_ret = port_test1['retp'].mean()*252
print(annual_ret)

annual_excess_ret = port_test1['Excess Return'].mean()*252
print(annual_excess_ret)

annual_std = port_test1['Excess Return'].std()*np.sqrt(252)
print(annual_std)

annual_sr = annual_excess_ret/annual_std
print(annual_sr)

0.14616262019078602
0.2556252733191254
0.2074678812784819
0.17754215407599863
1.1685556163167496


# 4.2 Case 2 Growth + Vol

In [74]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)

# RANKING
df_ins['rank'] = df_ins['growth rank'] + df_ins['volatility rank']
df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]

#Get weight and return data
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port_test2 = top1000['retp'].groupby('DATE').sum()
port_test2 = pd.DataFrame(port_test2)
port_test2 = port_test2.merge(ff, how = 'left', on = 'DATE')
port_test2['Excess Return'] = port_test2['retp'] - port_test2['RF']

from sklearn.linear_model import LinearRegression
ffmodel = LinearRegression()
X = port_test2[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port_test2['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
print(annual_alpha)

#Find performances
annual_ret = port_test2['retp'].mean()*252
print(annual_ret)

annual_excess_ret = port_test2['Excess Return'].mean()*252
print(annual_excess_ret)

annual_std = port_test2['Excess Return'].std()*np.sqrt(252)
print(annual_std)

annual_sr = annual_excess_ret/annual_std
print(annual_sr)

0.12235853189252761
0.2290285790960321
0.1812027233057918
0.15885856801987291
1.1406543919187517


# 4.3 Profit + Vol

In [75]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)

#RANKING
df_ins['rank'] = df_ins['profitability rank'] + df_ins['volatility rank']

df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]

#Get weight and return data
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port_test3 = top1000['retp'].groupby('DATE').sum()
port_test3 = pd.DataFrame(port_test3)
port_test3 = port_test3.merge(ff, how = 'left', on = 'DATE')
port_test3['Excess Return'] = port_test3['retp'] - port_test3['RF']

from sklearn.linear_model import LinearRegression
ffmodel = LinearRegression()
X = port_test3[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port_test3['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
print(annual_alpha)

#Find performances
annual_ret = port_test3['retp'].mean()*252
print(annual_ret)

annual_excess_ret = port_test3['Excess Return'].mean()*252
print(annual_excess_ret)

annual_std = port_test3['Excess Return'].std()*np.sqrt(252)
print(annual_std)

annual_sr = annual_excess_ret/annual_std
print(annual_sr)

0.08549129151295466
0.19139900032001694
0.14357314452977657
0.15410553005769467
0.9316547204764491


# 4.4 Growth

In [76]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)

#RANKING
df_ins['rank'] = df_ins['growth rank']
df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]

#Get weight and return data
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port_test4 = top1000['retp'].groupby('DATE').sum()
port_test4 = pd.DataFrame(port_test4)
port_test4 = port_test4.merge(ff, how = 'left', on = 'DATE')
port_test4['Excess Return'] = port_test4['retp'] - port_test4['RF']

from sklearn.linear_model import LinearRegression
ffmodel = LinearRegression()
X = port_test4[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port_test4['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
print(annual_alpha)

#Find performances
annual_ret = port_test4['retp'].mean()*252
print(annual_ret)

annual_excess_ret = port_test4['Excess Return'].mean()*252
print(annual_excess_ret)

annual_std = port_test4['Excess Return'].std()*np.sqrt(252)
print(annual_std)

annual_sr = annual_excess_ret/annual_std
print(annual_sr)

0.18867014655008202
0.29765876715542683
0.24950137511478332
0.1823087173615596
1.3685652486927733


# 4.5 Profit

In [77]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)

#RANKING
df_ins['rank'] = df_ins['profitability rank']
df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]

#Get weight and return data
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port_test5 = top1000['retp'].groupby('DATE').sum()
port_test5 = pd.DataFrame(port_test5)
port_test5 = port_test5.merge(ff, how = 'left', on = 'DATE')
port_test5['Excess Return'] = port_test5['retp'] - port_test5['RF']

from sklearn.linear_model import LinearRegression
ffmodel = LinearRegression()
X = port_test5[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port_test5['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
print(annual_alpha)

#Find performances
annual_ret = port_test5['retp'].mean()*252
print(annual_ret)

annual_excess_ret = port_test5['Excess Return'].mean()*252
print(annual_excess_ret)

annual_std = port_test5['Excess Return'].std()*np.sqrt(252)
print(annual_std)

annual_sr = annual_excess_ret/annual_std
print(annual_sr)

0.12152856512073887
0.22996980528444103
0.18181241324379754
0.177645536666986
1.0234561287324813


# 4.6 Volatility

In [78]:
# High P/B ratio
df_ins['growth rank'] = df_ins['PB'].groupby('DATE').rank(ascending=False)
# High Income/Asset
df_ins['profitability rank'] = df_ins['lag profitability'].groupby('DATE').rank(ascending=False)
# Low Volatility
df_ins['volatility rank'] = df_ins['volatility'].groupby('DATE').rank(ascending=True)

#RANKING
df_ins['rank'] = df_ins['volatility rank']
df_ins['rank'] = df_ins['rank'].groupby('DATE').rank()
top1000 = df_ins.loc[df_ins['rank'] < 1000]

#Get weight and return data
top1000['wt'] = top1000['MV'] / top1000['MV'].groupby('DATE').sum()
top1000['retp'] = top1000['RET'] * top1000['wt']
port_test6 = top1000['retp'].groupby('DATE').sum()
port_test6 = pd.DataFrame(port_test6)
port_test6 = port_test6.merge(ff, how = 'left', on = 'DATE')
port_test6['Excess Return'] = port_test6['retp'] - port_test6['RF']

from sklearn.linear_model import LinearRegression
ffmodel = LinearRegression()
X = port_test6[['Mkt-RF', 'SMB', 'HML']]
x = np.array(X).reshape(-1, 3)
y = np.array(port_test6['Excess Return'])
ffmodel.fit(x, y)

# Find factor loadings 
ff_betas = ffmodel.coef_
ff_betas

# Find annualized alpha under the 3-factor model
annual_alpha = ffmodel.intercept_ * 252
print(annual_alpha)

#Find performances
annual_ret = port_test6['retp'].mean()*252
print(annual_ret)

annual_excess_ret = port_test6['Excess Return'].mean()*252
print(annual_excess_ret)

annual_std = port_test6['Excess Return'].std()*np.sqrt(252)
print(annual_std)

annual_sr = annual_excess_ret/annual_std
print(annual_sr)

0.06292824765430846
0.16991721665253207
0.12209136086229172
0.14423144807497923
0.8464961178149033
